In [1]:
import tensorflow as tf
import numpy as np

In [2]:
cnf = np.asarray(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]]
       ])
sol = np.asarray([[1, 1], [-1, -1]])

In [3]:
VARIABLE_NUM = 2
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 2
LEN = 2

In [4]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [ ]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE))
        self.labels = tf.placeholder(tf.float32, shape=(batch_size, VARIABLE_NUM))
        
        vars_ = tf.abs(self.inputs) - 1
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [batch_size, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [batch_size, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        for timestep in tf.unstack(clause_embeddings, num=LEN, axis=1):
            formula_embedding, state = lstm(timestep, state)
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, VARIABLE_NUM]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([VARIABLE_NUM]), name='softmax_b')
        
        self.logits = formula_embedding @ softmax_w + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy((self.labels + 1) / 2, self.logits) 
        self.probabilities = tf.sigmoid(self.logits) * 2 - 1

model = Graph()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    logits, probs = sess.run([model.logits, model.probabilities], feed_dict={
        model.inputs: cnf
    })
    
print(logits)
print(probs)

In [ ]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer().minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(1000):
        sess.run(train_op, feed_dict={
             model.inputs: cnf, model.labels: sol
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: cnf
    })
    
probs